In [25]:
import numpy as np
import pandas as pd
import tensorflow as tf
import gensim

from tensorflow.python.keras.models import Model
from tensorflow.python.keras.layers import Input, Dense, GRU, Embedding
from tensorflow.python.keras.optimizers import RMSprop
from tensorflow.python.keras.callbacks import EarlyStopping, ModelCheckpoint, TensorBoard
from tensorflow.python.keras.preprocessing.text import Tokenizer
from tensorflow.python.keras.preprocessing.sequence import pad_sequences

import os
print(os.listdir('../input'))

ModuleNotFoundError: No module named 'gensim'

In [9]:
tf.__version__

'1.12.0'

In [10]:
tf.keras.__version__

'2.1.6-tf'

In [4]:
df = pd.read_csv('../input/train.csv')

In [7]:
df[['question_text', 'target']]

,question_text,target
0,How did Quebec nationalists see their province...,0
1,"Do you have an adopted dog, how would you enco...",0
2,Why does velocity affect time? Does velocity a...,0
3,How did Otto von Guericke used the Magdeburg h...,0
4,Can I convert montra helicon D to a mountain b...,0
5,"Is Gaza slowly becoming Auschwitz, Dachau or T...",0
6,Why does Quora automatically ban conservative ...,0
7,Is it crazy if I wash or wipe my groceries off...,0
8,"Is there such a thing as dressing moderately, ...",0
9,Is it just me or have you ever been in this ph...,0


In [14]:
data_src = df['question_text']
targets = df['target']

In [15]:
data_src[0]

'How did Quebec nationalists see their province as a nation in the 1960s?'

In [16]:
targets[0]

0

In [17]:
data_src[22]

'Has the United States become the largest dictatorship in the world?'

In [18]:
targets[22]

1

In [11]:
mark_start = 'ssss '
mark_end = ' eeee'

In [12]:
num_words = 80000 # tmp

In [ ]:
# load embedding
model = gensim.models.Word2Vec.load_word2vec_format('./model/GoogleNews-vectors-negative300.bin', binary=True)  

In [13]:
class TokenizerWrap(Tokenizer):
    
    def __init__(self, texts, padding,
                 reverse=False, num_words=None):

        Tokenizer.__init__(self, num_words=num_words)

        # Create the vocabulary from the texts.
        self.fit_on_texts(texts)

        # Create inverse lookup from integer-tokens to words.
        self.index_to_word = dict(zip(self.word_index.values(),
                                      self.word_index.keys()))
        # Convert all texts to lists of integer-tokens.
        # Note that the sequences may have different lengths.
        self.tokens = self.texts_to_sequences(texts)

        if reverse:
            # Reverse the token-sequences.
            self.tokens = [list(reversed(x)) for x in self.tokens]
        
            # Sequences that are too long should now be truncated
            # at the beginning, which corresponds to the end of
            # the original sequences.
            truncating = 'pre'
        else:
            # Sequences that are too long should be truncated
            # at the end.
            truncating = 'post'

        # The number of integer-tokens in each sequence.
        self.num_tokens = [len(x) for x in self.tokens]
                # Max number of tokens to use in all sequences.
        # We will pad / truncate all sequences to this length.
        # This is a compromise so we save a lot of memory and
        # only have to truncate maybe 5% of all the sequences.
        self.max_tokens = np.mean(self.num_tokens) \
                          + 2 * np.std(self.num_tokens)
        self.max_tokens = int(self.max_tokens)

        # Pad / truncate all token-sequences to the given length.
        # This creates a 2-dim numpy matrix that is easier to use.
        self.tokens_padded = pad_sequences(self.tokens,
                                           maxlen=self.max_tokens,
                                           padding=padding,
                                           truncating=truncating)

    def token_to_word(self, token):

        word = " " if token == 0 else self.index_to_word[token]
        return word 
    def tokens_to_string(self, tokens):

        # Create a list of the individual words.
        words = [self.index_to_word[token]
                 for token in tokens
                 if token != 0]
        
        text = " ".join(words)

        return text
    def text_to_tokens(self, text, reverse=False, padding=False):

        tokens = self.texts_to_sequences([text])
        tokens = np.array(tokens)

        if reverse:

            tokens = np.flip(tokens, axis=1)

            truncating = 'pre'
        else:
            truncating = 'post'
        if padding:
            tokens = pad_sequences(tokens,
                                   maxlen=self.max_tokens,
                                   padding='pre',
                                   truncating=truncating)

        return tokens

In [19]:
%%time
tokenizer_src = TokenizerWrap(texts=data_src,
                              padding='pre',
                              reverse=False,
                              num_words=num_words)

CPU times: user 47.9 s, sys: 240 ms, total: 48.1 s
Wall time: 48.1 s


In [21]:
tokens_src = tokenizer_src.tokens_padded
print(tokens_src.shape)

(1306122, 26)


In [24]:
data_src[22]

'Has the United States become the largest dictatorship in the world?'

In [23]:
tokens_src[22]

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,   58,    1,  344,  277,   93,    1, 1599,
       6108,    6,    1,   94], dtype=int32)